In [1]:
from finlab.crawler import table_date_range, update_table, to_pickle, out, commit,merge, date_range
import datetime
from io import StringIO
import requests
from bs4 import BeautifulSoup
import time

import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#import pandas_datareader.data as web

import os
from openpyxl import load_workbook

In [2]:
df = pd.read_pickle("history/tables/bargin_report_new.pkl")



In [3]:
print(df.shape) #org (142831, 15)

#print(df.index.names)
#print(df.tail())
print(df.sort_index(level='date', ascending=True).head(1).index.get_level_values('date'))
print(df.sort_index(level='date', ascending=True).tail(1).index.get_level_values('date'))
df[df.index.get_level_values('stock_id')=="1101 台泥"].head()
df[df.index.get_level_values('stock_id')=="1101 台泥"].tail()


(1417976, 15)
DatetimeIndex(['2012-05-02'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2020-06-12'], dtype='datetime64[ns]', name='date', freq=None)


外陸資買進股數(不含外資自營商) 外陸資賣出股數(不含外資自營商) 外陸資買賣超股數(不含外資自營商)  \
stock_id date                                                             
1101 台泥  2020-06-08          6213000          3694848           2518152   
         2020-06-09          3580046          6129148          -2549102   
         2020-06-10         14379207          6745767           7633440   
         2020-06-11          6428842          6964313           -535471   
         2020-06-12         10355733         11410996          -1055263   

                    外資自營商買進股數 外資自營商賣出股數 外資自營商買賣超股數  投信買進股數  投信賣出股數  投信買賣超股數  \
stock_id date                                                                 
1101 台泥  2020-06-08       0.0       0.0        0.0  321000  185000   136000   
         2020-06-09       0.0       0.0        0.0    2000  669000  -667000   
         2020-06-10       0.0       0.0        0.0   41000   16000    25000   
         2020-06-11       0.0       0.0        0.0       0  476000  -476000   
         2020-06-12       0.0       0.0        0.0  604000  814000  -210000   

                    自營商買進股數(自行買賣) 自營商賣出股數(自行買賣) 自營商買賣超股數(自行買賣) 自營商買進股數(避險)  \
stock_id date                                                                
1101 台泥  2020-06-08          4000        110000        -106000       68000   
         2020-06-09             0        150000        -150000       53000   
         2020-06-10         32000        559000        -527000       28000   
         2020-06-11             0        248000        -248000           0   
         2020-06-12        896000         82000         814000           0   

                    自營商賣出股數(避險) 自營商買賣超股數(避險)  
stock_id date                                 
1101 台泥  2020-06-08       60000         8000  
         2020-06-09       66000       -13000  
         2020-06-10      131000      -103000  
         2020-06-11      165000      -165000  
         2020-06-12      153000      -153000

In [ ]:

def expand_bargin20171218_20190211(date):
    
    URL = 'https://www.twse.com.tw/fund/T86?response=csv&date='+date.strftime("%Y%m%d")+'&selectType=ALLBUT0999'
    res = requests.get(URL)
    time.sleep(5)
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    df1 = pd.read_csv(StringIO(res.text.replace("=","")), header=1)
    time.sleep(5)
    df1 = df1.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df1=df1[df1['證券名稱'].notna()]

    id=[]

    for i in range(len(df1)):
        id.append(df1['證券代號'][i].replace(' ','')+' '+df1['證券名稱'][i].replace(' ',''))
    #print(id)

    df1['stock_id']=id
    df1['date']=date

    df1=df1.set_index(['stock_id', 'date'])
    df1=df1.drop(columns=['證券代號', '證券名稱','自營商買賣超股數','三大法人買賣超股數'])
    df1 = df1.apply(lambda s: s.astype(str).str.replace(",", ""))
    print(df1.sort_index(level='date', ascending=True).head(1).index.get_level_values('date'))
    return df1

def expand_bargin20141201_20171215(date):
    
    URL = 'https://www.twse.com.tw/fund/T86?response=csv&date='+date.strftime("%Y%m%d")+'&selectType=ALLBUT0999'
    res = requests.get(URL)
    time.sleep(5)
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    df1 = pd.read_csv(StringIO(res.text.replace("=","")), header=1)
    time.sleep(5)
    df1 = df1.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df1=df1[df1['證券名稱'].notna()]

    id=[]

    for i in range(len(df1)):
        id.append(df1['證券代號'][i].replace(' ','')+' '+df1['證券名稱'][i].replace(' ',''))
    #print(id)

    df1['stock_id']=id
    df1['date']=date

    df1=df1.set_index(['stock_id', 'date'])
    df1=df1.drop(columns=['證券代號', '證券名稱','自營商買賣超股數','三大法人買賣超股數'])
    df1=df1.rename(columns={"外資買進股數": "外陸資買進股數(不含外資自營商)", "外資賣出股數": "外陸資賣出股數(不含外資自營商)","外資買賣超股數":"外陸資買賣超股數(不含外資自營商)"})
    df1 = df1.apply(lambda s: s.astype(str).str.replace(",", ""))
    print(df1.sort_index(level='date', ascending=True).head(1).index.get_level_values('date'))
    return df1


def expand_bargin20120502_20141128(date):
    
    URL = 'https://www.twse.com.tw/fund/T86?response=csv&date='+date.strftime("%Y%m%d")+'&selectType=ALLBUT0999'
    res = requests.get(URL)
    time.sleep(5)
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    df1 = pd.read_csv(StringIO(res.text.replace("=","")), header=1)
    time.sleep(5)
    df1 = df1.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df1=df1[df1['證券名稱'].notna()]

    id=[]

    for i in range(len(df1)):
        id.append(df1['證券代號'][i].replace(' ','')+' '+df1['證券名稱'][i].replace(' ',''))
    #print(id)

    df1['stock_id']=id
    df1['date']=date

    df1=df1.set_index(['stock_id', 'date'])
    df1=df1.drop(columns=['證券代號', '證券名稱','三大法人買賣超股數'])
    df1=df1.rename(columns={"外資買進股數": "外陸資買進股數(不含外資自營商)", "外資賣出股數": "外陸資賣出股數(不含外資自營商)",
                            "外資買賣超股數":"外陸資買賣超股數(不含外資自營商)","自營商買進股數":"自營商買進股數(自行買賣)",
                            "自營商賣出股數":"自營商賣出股數(自行買賣)","自營商買賣超股數":"自營商買賣超股數(自行買賣)"})    
    
    df1 = df1.apply(lambda s: s.astype(str).str.replace(",", ""))
    print(df1.sort_index(level='date', ascending=True).head(1).index.get_level_values('date'))
    return df1

#df_combined=pd.DataFrame()
#days=pd.date_range(start="2017-12-18",end="2019-02-11")
#for date in days:
#    df_tmp=expand_bargin20171218_20190211(date)
#    df_combined=df_combined.append(df_tmp)
#    time.sleep(10)


In [ ]:
df_combined=pd.DataFrame()
days=pd.date_range(start="2012-05-02",end="2012-06-30")
for date in days:
    df_tmp=expand_bargin20120502_20141128(date)
    df_combined=df_combined.append(df_tmp)
    time.sleep(10)
#print(df_combined[df_combined.index.get_level_values('stock_id')=="1101 台泥"].head())

In [ ]:
df_combined[df_combined.index.get_level_values('stock_id')=="1101 台泥"].head(200)

In [ ]:
df = pd.read_pickle("history/tables/bargin_report_new.pkl")
df=df.append(df_combined)
df=df.sort_index()

df.drop_duplicates(keep = 'first', inplace = True) 
print(df[df.index.get_level_values('stock_id')=="1101 台泥"].head())
print(df[df.index.get_level_values('stock_id')=="1101 台泥"].tail())
to_pickle(df, 'bargin_report_new')

In [24]:
df = pd.read_pickle("history/tables/monthly_report_new.pkl")
df[(df.index.get_level_values('stock_id')=="2330 台積電") & (df.index.get_level_values('date')>"2004-01-01")]



當月營收       上月營收     去年當月營收 上月比較增減(%) 去年同月增減(%)  \
stock_id date                                                              
2330 台積電 2004-01-10   18970000        NaN        NaN      2.46      68.2   
         2004-02-10   19160000        NaN        NaN         1      45.9   
         2004-03-10   18390000        NaN        NaN     -4.02      48.9   
         2004-04-10   19970000        NaN        NaN      8.63      44.2   
         2004-05-10   20630000        NaN        NaN       3.3      35.2   
         2004-06-10   21720000        NaN        NaN      5.28      29.2   
         2004-07-10   22520000        NaN        NaN      3.68      26.2   
         2004-08-10   23150000        NaN        NaN      2.79      31.2   
         2004-09-10   23410000        NaN        NaN      1.15      27.8   
         2004-10-10   23170000        NaN        NaN     -1.03      22.5   
         2004-11-10   22970000        NaN        NaN     -0.88      13.1   
         2004-12-10   20990000        NaN        NaN     -8.64      13.4   
         2005-01-10   19920000        NaN        NaN     -5.08      5.03   
         2005-02-10   20842779   19919339   19155894      4.63       8.8   
         2005-03-10   17200047   20842779   18385262    -17.47     -6.44   
         2005-04-10   17610390   17200047   19972049      2.38    -11.82   
         2005-05-10   18903406   17610390   20630606      7.34     -8.37   
         2005-06-10   19507661   18903406   21719561      3.19    -10.18   
         2005-07-10   20105150   19507661   22519074      3.06    -10.71   
         2005-08-10   20847066   20105150   23147633      3.69     -9.93   
         2005-09-10   23182037   20847066   23413876      11.2     -0.99   
         2005-10-10   25228512   23182037   23173529      8.82      8.86   
         2005-11-10   26227155   25228512   22969780      3.95     14.18   
         2005-12-10   27518437   26227155   20985825      4.92     31.12   
         2006-01-10   27415724   27518437   19919338     -0.37     37.63   
         2006-02-10   26177607   27415724   20842779     -4.51     25.59   
         2006-03-10   24008853   26177607   17200048     -8.28     39.58   
         2006-04-10   27106885   24008853   17610390      12.9     53.92   
         2006-05-10   27162375   27106885   18903406       0.2     43.69   
         2006-06-10   26770174   27162375   19507661     -1.44     37.22   
...                        ...        ...        ...       ...       ...   
         2017-11-10   94519722   88578988   91085318       6.7      3.77   
         2017-12-10   93153310   94519722   93029912     -1.44      0.13   
         2018-01-10   89897252   93153310   78112022     -3.49     15.08   
         2018-02-10   79740672   89897252   76616258    -11.29      4.07   
         2018-03-10   64640562   79740672   71422961    -18.93     -9.49   
         2018-04-10  103697437   64640562   85875181     60.42     20.75   
         2018-05-10   81869781  103697437   56871933    -21.04     43.95   
         2018-06-10   80968732   81869781   72795974      -1.1     11.22   
         2018-07-10   70438298   80968732   84187305     -13.0    -16.33   
         2018-08-10   74370924   70438298   71611190      5.58      3.85   
         2018-09-10   91055038   74370924   91917167     22.43     -0.93   
         2018-10-10   94921920   91055038   88578988      4.24      7.16   
         2018-11-10  101550181   94921920   94519722      6.98      7.43   
         2018-12-10   98389414  101550181   93153310     -3.11      5.62   
         2019-01-10   89830598   98389414   89897252     -8.69     -0.07   
         2019-02-10   78093827   89830598   79740672    -13.06     -2.06   
         2019-03-10   60889055   78093827   64640562    -22.03      -5.8   
         2019-04-10   79721587   60889055  103697437     30.92    -23.12   
         2019-05-10   74693615   79721587   81869781      -6.3     -8.76   
         2019-06-10   80436931   74693615   80968732      7.68     -0.65  